<a href="https://colab.research.google.com/github/informatics-isi-edu/eye-ai-exec/blob/main/notebooks/VGG19_Diagnosis_Predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# VGG19 Model Application

This notebook applied a pre-trained model to a dataset specified in the configuration file and uploads the labels to the catalog.  The ROC curve is also calculated and uploaded.

In [4]:
# Prerequisites to configure colab
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !pip install deriva
    !pip install bdbag
    !pip install --upgrade --force pydantic
    !pip install git+https://github.com/informatics-isi-edu/deriva-ml git+https://github.com/informatics-isi-edu/eye-ai-ml
    !pip install setuptools_git_versioning


In [5]:
repo_dir = "Repos"   # Set this to be where your github repos are located.
%load_ext autoreload
%autoreload 2

# Update the load path so python can find modules for the model
import sys
from pathlib import Path
sys.path.insert(0, str(Path.home() / repo_dir / "eye-ai-ml"))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import json
import os
import eye_ai
from eye_ai.eye_ai import EyeAI
import pandas as pd
from pathlib import Path, PurePath
import logging
print(f"Using eye_ai version: {eye_ai.__version__}")
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)

Using eye_ai version: 1.1.0.post5+git.b32c4a27


In [7]:
# @title login to DERIVA via Globus Auth

DEBUG_MODE = False #@param ["False", "True"] {type:"raw"}
catalog_id = "eye-ai" #@param
DEFAULT_SERVER = 'dev.eye-ai.org' if DEBUG_MODE else 'www.eye-ai.org'

!deriva-globus-auth-utils login --no-browser --host {DEFAULT_SERVER}


You are already logged in.


Connect to Eye-AI catalog.  Configure to store data local cache and working directories.  Initialize Eye-AI for pending execution based on the provided configuration file.

In [ ]:
# Variables to configure the rest of the notebook.

cache_dir = '/content/data'        # Directory in which to cache materialized BDBags for datasets
working_dir = '/content/working'    # Directory in which to place output files for later upload.

configuration_rid="2-A5KC"      # Configuration file for this run.  Needs to be changed for each execution.

In [ ]:
EA = EyeAI(hostname = DEFAULT_SERVER, catalog_id = catalog_id, cache_dir= cache_dir, working_dir=working_dir)


In [ ]:
# @title Initiate an Execution
configuration_records = EA.execution_init(configuration_rid=configuration_rid)
input_dataset = configuration_records.bag_paths[0] # Assumes that the configuration file only specifies one dataset.
configuration_records.model_dump()

Algorithm was trained on cropped images, so take the raw images and bounding boxes and apply, storing the results in the working directory.

In [ ]:
# @title Get Cropped Images
cropped_image_path, cropped_csv = EA.create_cropped_images(str(configuration_records.bag_paths[0]),
                                                           output_dir = working_dir,
                                                           crop_to_eye=False)

Import the actual model code and then run against the input dataset specified in the configuration file.  

In [ ]:
# @title Execute Proecss algorithm (Test model)
from eye_ai.models.vgg19_diagnosis_predict import prediction

with EA.execution(execution_rid=configuration_records.execution_rid) as exec:
  output_path = EA.execution_assets_path/Path("Model_Prediction")
  pred_csv_path = prediction(input_dataset, cropped_image_path, output_path)

In [ ]:
# @title Plot ROC.
roc_value_path = EA.plot_roc(pred_csv_path)

Add the new lables to the catalog using the provided diagnosis tage for this execution.  Also upload any additional assets that were produced by this execution..

In [ ]:
# @title Save Diagnosis
import re
pred_df = pd.read_csv(pred_csv_path)
pred_df['Image'] = pred_df['Filename'].apply(lambda x: re.search(r'Cropped_(.*?)\.', x).group(1))
# The input dataframe need two columns: Image (containing image rid) and Prediction (containing 0/1)
EA.insert_new_diagnosis(pred_df,
                        configuration_records.vocabs['Diagnosis_Tag'][0].rid,
                        configuration_records.execution_rid)


In [ ]:
# @title Save Execution Assets (model) and Metadata
uploaded_assets = EA.execution_upload(configuration_records.execution_rid, False)
